In [14]:
%load_ext autoreload
%autoreload 2

In [16]:
import json
import torchaudio
from pathlib import Path
from copy import copy
from docx.api import Document
import soundfile as sf
import librosa
from pydub.utils import mediainfo
from tqdm.auto import tqdm
import pandas as pd

from data_utils import NeutuneSet, OnFlyAudio

In [2]:
# dataset = NeutuneSet('/home/teo/userdata/nia_dataset/field_recording')
# dataset = NeutuneSet('/home/teo/label-studio_files/nia_dataset/')
dataset = NeutuneSet('/home/clay/userdata/nia_dataset/data_v4/01/')

In [44]:
from pandas import read_table

from meta_utils import MetaCreator

meta_creator = MetaCreator('vocab.json')
# meta_creator.create_meta_for_wav(dataset.wav_list[0])
entire_meta = [meta_creator.create_meta_for_wav(wav) for wav in tqdm(sorted(dataset.wav_list))]

  0%|          | 0/497 [00:00<?, ?it/s]

In [45]:
error_list = meta_creator.error_list
dict_list = meta_creator.error_dict
print(len(error_list))
dict_list

72


defaultdict(list,
            {'Meta docx does not exist': ['fs-2-4-75-1-김양우.wav',
              'fs-2-4-75-2-김양우.wav',
              'fs-2-4-75-3-김양우.wav',
              'fs-2-4-78-1-김양우.wav',
              'fs-2-4-78-2-김양우.wav',
              'fs-2-4-78-3-김양우.wav',
              'fs-2-4-86-1-김양우.wav',
              'fs-2-4-86-2-김양우.wav',
              'fs-2-4-86-3-김양우.wav',
              'fs-2-4-87-1-김양우.wav',
              'fs-2-4-87-2-김양우.wav',
              'fs-2-4-87-3-김양우.wav',
              'r-1-1-2-3 허진만.wav'],
             'Error occured while handling the corresponding docx': ['fs-2-4-75-1-김준형.wav',
              'fs-2-4-75-1-이동화.wav',
              'fs-2-4-75-1-장완석.wav',
              'fs-2-4-75-2-김준형.wav',
              'fs-2-4-75-2-이동화.wav',
              'fs-2-4-75-2-장완석.wav',
              'fs-2-4-75-3-김준형.wav',
              'fs-2-4-75-3-이동화.wav',
              'fs-2-4-75-3-장완석.wav',
              'fs-2-4-78-1-김준형.wav',
              'fs-2-4-78-1-송화영.wav',
            

In [112]:
docx_path = '/home/clay/userdata/nia_dataset/data_v4/01/metadata/r-1-1-1-2-김영운.docx'
meta_creator.read_table_from_docx(docx_path)

IndexError: tuple index out of range

In [110]:
count = 0

document = Document(docx_path)
table = document.tables[0]
for row in table.rows:
  print(row)
  try:
    print(row.cells[2].text)
  except:
    continue
  
  '''
  if count != 0:
    print(row.cells)
  count += 1
  '''


분류 번호
마곡 코트야드 호텔 뷔페 옆 큰 공간
녹음 연/월/일
Zoom H6 레코더, 윈드쉴드, 헤드셋
오디오 길이
서울시 강서구 마곡중앙 12로
음원으로부터의 거리
약간 흐림
보안 이슈
호텔 음악소리 살짝 섞임


In [116]:
excel_path = '/home/clay/userdata/nia_dataset/data_v4/01/metadata/r-1-1-1-1-정욱.xlsx'
detailed_meta = pd.read_excel(excel_path, index_col=0)
detailed_meta

,Unnamed: 1,Unnamed: 2,Unnamed: 3
1차 원시데이터 기록표 (필드레코딩),,,
번호,구분,내용,비고
1,분류 표기,1-1-1,NaN
2,상황 묘사(제목),해밀톤 호텔 1층 로비,NaN
3,녹음 장소,서울특별시 용산구 이태원로179 해밀톤관광호텔,NaN
4,녹음 지점,"37.5347253, 126.993847",NaN
5,기후/환경 조건,비 많이 옴,NaN
7,오디오 포맷,"wav (스테레오, 96k, 24bit)",NaN
8,사용 기자재,"Zoom H6 레코더, 윈드실드, 인이어",NaN
9,녹음/제작자 성명,정욱,NaN


In [118]:
excel_path = '/home/clay/userdata/nia_dataset/data_v4/01/metadata/fs-2-4-78-2-어상진.xlsx'
detailed_meta = pd.read_excel(excel_path, index_col=0)
detailed_meta

,Unnamed: 1,Unnamed: 2,Unnamed: 3
1차 원시데이터 기록표 (폴리/합성),,,
번호,구분,내용,비고
1,분류 표기,1-3-24,분류 기준표 참고
2,상황 묘사(제목),에어컨 실외기 돌아가는 소리,구체적인 상황 기재
3,합성 방식 1,NaN,폴리 사운드 미활용시 공란 기재
4,제작시 사용 개체,NaN,폴리 방식 활용시 구체적 수음 방식 기재
5,합성 방식 2,"신디사이저 및 샘플, 이펙터를 이용한 합성 음원",합성 사운드 미활용시 공란 기재
6,제작시 사용 개체,Logic Pro DAW,가급적 상세 기재
8,오디오 포맷,"wav (스테레오, 96k, 24bit)",NaN
9,녹음/제작자 성명,어상진,NaN


In [115]:
excel_path = '/home/clay/userdata/nia_dataset/data_v4/01/metadata/fs-2-4-75-1-김수환.xlsx'
detailed_meta = pd.read_excel(excel_path, index_col=0)
detailed_meta

,Unnamed: 1
분류 번호,1(공간/환경)-1(항공기)-002(제트기)
제목,제트 엔진 소리
녹음/제작자 성명,김수환
녹음/제작 연/월/일,2022.07.17.
사용 기자재,Ableton Live DAW
오디오 포맷,"스테레오 - wav (96k, 24bit) / mp3"
오디오 길이,105초
제작 장소,서울시 금호로 107
보안 이슈,없음
자료 제공자 정보,Soundly


In [17]:
event_id = 0
dataset = OnFlyAudio('/home/clay/userdata/nia_dataset/data_v4/01/')
dataset.path, dataset.mp3_list[0]

(PosixPath('/home/clay/userdata/nia_dataset/data_v4/01'),
 PosixPath('/home/clay/userdata/nia_dataset/data_v4/01/mp3/fs-2-4-75-1-가민지.mp3'))

In [19]:
event_id = 0
sample_path = dataset.mp3_list[event_id].relative_to(dataset.path)
print(sample_path)
sample_path = '01' / sample_path
sample_path

mp3/fs-2-4-75-1-가민지.mp3


PosixPath('01/mp3/fs-2-4-75-1-가민지.mp3')

In [ ]:
import numpy as np

class AudioFeatureExtractor:
  def __init__(self):
    pass 

  def get_audio_features(self, y:np.ndarray, sr:int):
    spec = self.get_spec(y, sr)
    
    spec_centroid = self.get_mean_and_std_of_audio_feature(spec, sr, librosa.feature.spectral_centroid, 'spectral centroid')
    
    spec_centroid.update(self.get_mean_and_std_of_audio_feature(spec, sr, librosa.feature.spectral_bandwidth, 'spectral bandwidth'))
    spec_centroid.update(self.get_mean_and_std_of_audio_feature(spec, sr, librosa.feature.spectral_contrast, 'spectral contrast'))
    spec_centroid.update(self.get_mean_and_std_of_audio_feature(spec, sr, librosa.feature.spectral_rolloff, 'spectral roll off'))
    spec_centroid.update(self.get_mean_and_std_of_audio_feature_wo_sr(y, librosa.feature.spectral_flatness, 'spectral flatness'))
    spec_centroid.update(self.get_mean_and_std_of_audio_feature_wo_sr(y, librosa.feature.zero_crossing_rate, 'zero crossing rate'))
    spec_centroid.update(self.get_mean_and_std_of_audio_feature_wo_sr(y, librosa.feature.rms, 'rms'))

    return spec_centroid
  
  def get_spec(self, y, sr):
    return np.abs(librosa.stft(y))
  
  def get_mean_and_std_of_audio_feature(self, y, sr, audio_feature, audio_feature_name:str):
    audio_features = audio_feature(S = y, sr = sr)
    return {f'{audio_feature_name} mean': np.mean(audio_features) , f'{audio_feature_name} std':np.std(audio_features) }
  
  def get_mean_and_std_of_audio_feature_wo_sr(self, y, audio_feature, audio_feature_name:str):
    audio_features = audio_feature(y = y)
    return {f'{audio_feature_name} mean': np.mean(audio_features) , f'{audio_feature_name} std':np.std(audio_features) }

  def read_soundfile(self, soundfile:sf.SoundFile):
    y = soundfile.read()
    y = y.mean(1)
    return y, soundfile.samplerate

  def __call__(self, soundfile:sf.SoundFile):
    y, sr = self.read_soundfile(soundfile)
    return self.get_audio_features(y, sr)

extractor = AudioFeatureExtractor()
ob = sf.SoundFile(dataset.wav_list[290])

# extractor(ob)

In [ ]:
example_meta = entire_meta[290]
example_meta['오디오 음향 특성'] = extractor(ob)

In [ ]:
with open("example_meta.json", 'w') as f:
  json.dump(example_meta, f, ensure_ascii=False)
# example_meta

In [ ]:
import pandas as pd
pd.DataFrame([(x['소분류'], x['녹음 방식'], x['파일 이름'], x['길이']) for x in entire_meta]).to_csv('length.csv')

In [ ]:
len(dataset.wav_list)

In [ ]:
[error['file'] for error in meta_creator.error_list if error['error'] == 'Error occured while handling the corresponding docx']

In [ ]:
meta_creator.error_list

In [ ]:
entire_meta[290]

In [ ]:
for meta in entire_meta:
  if '필드 레코딩 메타' in meta:
    print(meta['필드 레코딩 메타']['소리의 강도'])

In [ ]:
test = Document("/home/teo/userdata/nia_dataset/field_recording/5팀 이상화 1주차/metadata/1-2-5-1-김민규.docx")
table = test.tables[0]

for row in table.rows:
  print(row.cells[0].text, '//', row.cells[1].text)

In [ ]:
meta_creator.read_table_from_docx("/home/teo/userdata/nia_dataset/field_recording/4팀 김양우 1주차/metadata/1-1-3-1-한은광.docx")

In [ ]:
Path("/home/teo/userdata/nia_dataset/field_recording/5팀 이상화 1주차/metadata/1-2-7-6-백승렬.docx").exists()

In [ ]:
table = meta_creator.read_table_from_docx('1-1-1-1-송영재.docx')
table

In [ ]:
import pandas as pd
pd.DataFrame(table, index=['']).T.to_excel('1-1-1-1-송영재.xlsx')

In [ ]:
pd.read_excel('/home/teo/label-studio_files/nia_dataset/__DATA_v4/01/metadata/r-1-1-4-11-백승렬.xlsx')

In [ ]:
!pip install openpyxl

In [ ]:
meta_creator.vocab['idx2title']

In [ ]:
mediainfo("/home/teo/userdata/nia_dataset/field_recording/5팀 이상화 1주차/mp3/1-2-7-6-백승렬.mp3")

In [ ]:
import docx

def getText(filename):
    doc = docx.Document(filename)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    return '\n'.join(fullText)

test = getText('1-1-1-1-송영재.docx')

In [ ]:
from docx.api import Document

# Load the first table from your document. In your example file,
# there is only one table, so I just grab the first one.
document = Document('1-1-1-1-송영재.docx')
table = document.tables[0]

In [ ]:
data = []

keys = None
for i, row in enumerate(table.rows):
    text = (cell.text for cell in row.cells)

    # Establish the mapping based on the first row
    # headers; these will become the keys of our dictionary
    if i == 0:
        keys = tuple(text)
        continue

    # Construct a dictionary for this row, mapping
    # keys to values for this row
    row_data = dict(zip(keys, text))
    data.append(row_data)
data[0]['']